# Auth Weekly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import csv
from datetime import datetime, timedelta
from config import Config

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

weekly total before running time.

In [ ]:
%%sql stat_accounts_weekly_completed  <<
SELECT count(*) FILTER (WHERE type_code = 'BASIC') AS BASIC
,count(*) FILTER (WHERE type_code = 'PREMIUM') AS PREMIUM
FROM orgs WHERE status_code='ACTIVE'
AND date(created at time zone 'utc' at time zone 'pst') > date(current_date - 1 - interval '1 weeks')
AND date(created at time zone 'utc' at time zone 'pst') <= date(current_date - 1)

Save to CSV

In [ ]:
filename = os.path.join(os.getcwd(), r'data/')+'auth_weekly_stats_till_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d') +'.csv'

df = stat_accounts_weekly_completed.DataFrame()
with open(filename, 'w') as f:
    df.insert(0, "Weekly Number of New Accounts:", [''], True)
    df.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql stat_accounts_total_completed  <<
SELECT count(*) FILTER (WHERE type_code = 'BASIC') AS BASIC
,count(*) FILTER (WHERE type_code = 'PREMIUM') AS PREMIUM
FROM orgs WHERE status_code='ACTIVE'

In [ ]:
df = stat_accounts_total_completed.DataFrame()
with open(filename, 'a') as f:      
    f.write('\n\n')
    df.insert(0, "Total Number of Accounts:", [''], True)
    df.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql stat_accounts_login_option_weekly_completed  <<
SELECT COUNT(*) FILTER (WHERE a.login_source = 'BCEID') AS BCEID
,COUNT(*) FILTER (WHERE a.login_source = 'BCSC') AS BCSC
FROM orgs o, account_login_options a 
WHERE o.id=a.org_id 
AND o.status_code='ACTIVE' 
AND a.is_active='TRUE' 
AND date(o.created at time zone 'utc' at time zone 'pst') > date(current_date - 1 - interval '1 weeks')
AND date(o.created at time zone 'utc' at time zone 'pst') <= date(current_date - 1)  

In [ ]:
df = stat_accounts_login_option_weekly_completed.DataFrame()

df['bceid_%'] = round((df.bceid / (df.bceid+df.bcsc)).astype('float') * float(100), 2)
df['bcsc_%'] = round((df.bcsc / (df.bceid+df.bcsc)).astype('float') * float(100), 2)

with open(filename, 'a') as f:
    f.write('\n\n')
    df.insert(0, "New Accounts for BCeID and BCSC from " + datetime.strftime(datetime.now()-timedelta(8), '%Y-%m-%d') + " to " + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d') + ":", [''], True)
    df.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql stat_accounts_login_option_weekly_last_year_completed  <<
SELECT COUNT(*) FILTER (WHERE a.login_source = 'BCEID') AS BCEID
,COUNT(*) FILTER (WHERE a.login_source = 'BCSC') AS BCSC
FROM orgs o, account_login_options a 
WHERE o.id=a.org_id 
AND o.status_code='ACTIVE' 
AND a.is_active='TRUE' 
AND date(o.created at time zone 'utc' at time zone 'pst') > date(current_date - 1 - interval '1 years' - interval '1 weeks')
AND date(o.created at time zone 'utc' at time zone 'pst') <= date(current_date - 1 - interval '1 years')  

In [ ]:
df = stat_accounts_login_option_weekly_last_year_completed.DataFrame()

df['bceid_%'] = round((df.bceid / (df.bceid+df.bcsc)).astype('float') * float(100), 2)
df['bcsc_%'] = round((df.bcsc / (df.bceid+df.bcsc)).astype('float') * float(100), 2)

with open(filename, 'a') as f:
    f.write('\n\n')
    df.insert(0, "New Accounts for BCeID and BCSC last year from " + datetime.strftime(datetime.now()-timedelta(373), '%Y-%m-%d') + " to " + datetime.strftime(datetime.now()-timedelta(366), '%Y-%m-%d') + ":", [''], True)
    df.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql stat_accounts_login_option_pass_years_completed  <<
SELECT SUBSTRING(CAST(date_part('year', o.created) AS VARCHAR), 1, 4) AS YEAR, COUNT(*) FILTER (WHERE a.login_source = 'BCEID') AS BCEID
, COUNT(*) FILTER (WHERE a.login_source = 'BCSC') AS BCSC
FROM orgs o, account_login_options a 
WHERE o.id=a.org_id 
AND o.status_code='ACTIVE' 
AND a.is_active='TRUE'
AND date(o.created at time zone 'utc' at time zone 'pst')  > date(current_date - interval '20 years')
GROUP BY date_part('year', o.created)
ORDER BY date_part('year', o.created) DESC;

In [ ]:
df = stat_accounts_login_option_pass_years_completed.DataFrame()
df['bceid_%'] = round((df.bceid / (df.bceid+df.bcsc)).astype('float') * float(100), 2)
df['bcsc_%'] = round((df.bcsc / (df.bceid+df.bcsc)).astype('float') * float(100), 2)

arr = [] 
for i in range(len(df)-1): 
	arr.append('',) 
arr.append('') 

with open(filename, 'a') as f:      
    f.write('\n\n')
    df.insert(0, "Total Number of BCeID and BCSC Accounts by Year:", arr , True)
    df.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql stat_users_weekly_completed  <<
SELECT count(*) FILTER (WHERE login_source = 'BCEID') AS BCEID
,count(*) FILTER (WHERE login_source = 'BCSC') AS BCSC
FROM users
WHERE date(login_time at time zone 'utc' at time zone 'pst') > date(current_date - 1 - interval '1 weeks')
AND date(login_time at time zone 'utc' at time zone 'pst') <= date(current_date - 1)

In [ ]:
df = stat_users_weekly_completed.DataFrame()
with open(filename, 'a') as f:      
    f.write('\n\n')
    df.insert(0, "Weekly Number of Login Users:", [''], True)
    df.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql stat_users_total_completed  <<
SELECT count(*) FILTER (WHERE login_source = 'BCEID') AS BCEID
,count(*) FILTER (WHERE login_source = 'BCSC') AS BCSC
FROM users

In [ ]:
df = stat_users_total_completed.DataFrame()
with open(filename, 'a') as f:      
    f.write('\n\n')
    df.insert(0, "Total Number of Login Users:", [''], True)
    df.to_csv(f, sep=',', encoding='utf-8', index=False)